Import Requirements

In [1]:
import pyodbc
import re

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10)

from sqlalchemy import create_engine
from datetime import datetime, timedelta

Connection

In [2]:
user = 'huda'
password = 'Vancha12'
host = '127.0.0.1'
port = 1433
database = 'HRSystemDB'


def get_connection():
    return create_engine(
        url=f"mssql+pyodbc://{user}:{password}@{host}:{port}/{database}?driver=SQL Server",
    )

engine = get_connection()
conn = engine.connect()

In [3]:
tables = engine.table_names()

C:\Users\eats\AppData\Local\Temp\ipykernel_11504\863585613.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  tables = engine.table_names()


Transform Function

In [4]:
def pick_date(str_date: str):
    return str_date[:10]

# menghitung berapa lama pengalaman berdasarkan tanggal mulai dan selesai
def get_experience_time(col1: pd.Series, col2: pd.Series):
    col1.map(pd.to_datetime)
    col2.map(pd.to_datetime)
    
    print(col1)
    print(col2)

    return (col1-col2).dt.months

# membenarkan tahun yang tidak jelas di database, eg: 212 --> 2012 dan 15 --> 2015
def repair_years(str_date: str):
    print(str_date)
    str_date = str_date.split('-')
    
    year = str(int(str_date[0]))
    if len(year) == 3:
        year = year[0] + '0' + year[1:]
        print(year)
    elif len(year) == 2:
        year = '20' + year
        print(year)
    else:
        pass
        
    str_date[0] = year
    return '-'.join(str_date)

def years_to_zero(str_date: str):
    str_date = str_date.split('-')

    year = int(str_date[0])
    if year < 1990:
        return '0'
    elif year > 2023:
        return '0'
    
    return '-'.join(str_date)

def html_cleaner(text: str):
    re_html = re.compile(r'<[^>]+>')
    return re_html.sub('', text)

def remove_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]

# belum dipake
def clearing_nationality(text: str):
    text = text.strip().lower()

    text_dict = ('wni', 'indonesia', 'jawa', 'java')

    return 'indonesia' if text in text_dict else text

def get_age(dob):
    return (datetime.now() - dob) / np.timedelta64(1, 'Y')

def remove_badyears(str_date, min_border, max_border):
    year = str_date.split('-')[0]

    if max_border < int(year) or int(year) < min_border:
        return np.NaN
    return str_date

### Tables

In [5]:
df_applicant = pd.DataFrame(engine.execute('SELECT ApplicantID, DiseaseHistory, Dob, ExpectedSalary, IsAbleToWorkRemote, CityID, ProvinceID, DriverLicenseType, Gender, IsUsingGlasses, Height, MaritalStatus, Nationality, Strengthness, Weaknesses, TypeOfVehicle FROM Applicant'))
df_applicant_education = pd.DataFrame(engine.execute('SELECT ApplicantEducationID, ApplicantID, DateStart, DateEnd, EducationInstituteName, Score, EducationLevelID, MajorID, Degree FROM ApplicantEducation'))
df_applicant_experience = pd.DataFrame(engine.execute('SELECT ApplicantExperienceID, ApplicantID, DateFrom, DateTo, Industry, CompanyName, JobDescription, Position, Salary FROM ApplicantExperience'))
df_applicant_document = pd.DataFrame(engine.execute('SELECT ApplicantID, DocumentName FROM ApplicantDocument'))
df_applicant_certificate = pd.DataFrame(engine.execute('SELECT ApplicantID, Description FROM ApplicantCertificate'))

df_pipeline = pd.DataFrame(engine.execute('SELECT PipelineID, ApplicantID, JobID, StageID FROM Pipeline'))
df_stage = pd.DataFrame(engine.execute('SELECT StageID, Label FROM Stage'))

df_job = pd.DataFrame(engine.execute('SELECT * FROM Job'))
df_function_position = pd.DataFrame(engine.execute('SELECT FunctionPositionID, FunctionPositionName FROM FunctionPosition'))
df_department = pd.DataFrame(engine.execute('SELECT DepartmentID, Name FROM Department'))
df_city = pd.DataFrame(engine.execute('SELECT CityID, Name FROM City'))
df_province = pd.DataFrame(engine.execute('SELECT ProvinceID, Name FROM Province'))
df_major = pd.DataFrame(engine.execute('SELECT MajorID, MajorName FROM Major'))

In [8]:
df_function_position.to_csv('data/df_function_position.csv', index=['FunctionPositionID'])
df_department.to_csv('data/df_department.csv', index=['DepartmentID'])
df_city.to_csv('data/df_city.csv', index=['CityID'])
df_province.to_csv('data/df_province.csv', index=['ProvinceID'])
df_major.to_csv('data/df_major.csv', index=['MajorID'])

In [21]:
tes = pd.merge(df_applicant, df_pipeline, on=['ApplicantID'])
tes = pd.merge(tes, df_stage, on=['StageID'])
tes = tes[(tes.Label == 'Rejected') | (tes.Label == 'Hired')]

print('Data Rejected & Hired sebelum preprocessing:', tes.drop_duplicates(['ApplicantID']).Label.value_counts().values)

Data Rejected & Hired sebelum preprocessing: [8490  655]


In [617]:
# df_applicant.to_csv('data/df_applicant.csv')
# df_applicant_education.to_csv('data/df_applicant_education.csv')
# df_applicant_experience.to_csv('data/df_applicant_experience.csv')
# df_applicant_document.to_csv('data/df_applicant_document.csv')
# df_applicant_certificate.to_csv('data/df_applicant_certificate.csv')
# df_pipeline.to_csv('data/df_pipeline.csv')
# df_stage.to_csv('data/df_stage.csv')
# df_job.to_csv('data/df_job.csv')

# STEPS
# 1. Check duplicates
# 2. Group duplicates
# 3. Fillna
# 4. Lower
# 5. Int
# 6. 

Transform

In [607]:
df_applicant[df_applicant.ApplicantID.duplicated()]

,ApplicantID,DiseaseHistory,Dob,ExpectedSalary,IsAbleToWorkRemote,CityID,ProvinceID,DriverLicenseType,Gender,IsUsingGlasses,Height,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle


In [608]:
df_applicant.set_index('ApplicantID', inplace=True)

In [612]:
df_app_int = df_applicant.select_dtypes(int)
df_app_flt = df_applicant.select_dtypes(float).fillna(np.nan)
df_app_obj = df_applicant.select_dtypes(object).astype(str).fillna(np.nan)

In [616]:
pd.concat([df_app_int, df_app_flt, df_app_obj], axis=1)

,Height,ExpectedSalary,CityID,ProvinceID,DiseaseHistory,Dob,DriverLicenseType,Gender,IsUsingGlasses,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle
ApplicantID,,,,,,,,,,,,,,
1,0,0.0,NaN,NaN,,2018-04-10 09:55:35.1320927,,,False,,,,,
2,0,0.0,1.0,1.0,-,2018-04-10 00:00:00.0000000,A,Male,False,None,Indonesia,strength,weak,Mobil
3,178,4000000.0,1.0,1.0,none,1994-01-06 00:00:00.0000000,C,Male,False,Single,WNI,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Sepeda Motor
4,180,4000000.0,1.0,1.0,-,1995-05-25 00:00:00.0000000,C,Male,False,Single,Indonesia,cepat mengerti,kurang tegas,Sepeda Motor
5,0,0.0,NaN,NaN,,2018-04-11 07:22:05.6383519,,,False,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44927,0,0.0,NaN,NaN,,None,,,None,,,,,
44928,0,0.0,NaN,NaN,,None,,,None,,,,,
44929,0,0.0,NaN,NaN,,None,,,None,,,,,


In [575]:
df_applicant.isna().sum()

ApplicantID               0
DiseaseHistory            0
Dob                      37
ExpectedSalary            0
IsAbleToWorkRemote        0
CityID                26244
ProvinceID            25752
DriverLicenseType      2181
Gender                    0
IsUsingGlasses           40
Height                    0
MaritalStatus           256
Nationality             213
Strengthness           2751
Weaknesses             3139
TypeOfVehicle          1968
dtype: int64

In [ ]:
# df_applicant = df_applicant[~df_applicant['Dob'].isna()]

df_applicant['Dob'] = df_applicant['Dob'].fillna('')
df_applicant['Dob'] = df_applicant['Dob'].apply(pick_date)

df_applicant['CityID'] = df_applicant['CityID'].fillna(0)
df_applicant['CityID'] = df_applicant['CityID'].replace('', 0)
df_applicant['ProvinceID'] = df_applicant['ProvinceID'].fillna(0)

df_applicant['DriverLicenseType'] = df_applicant['DriverLicenseType'].fillna('')
df_applicant['IsUsingGlasses'] = df_applicant['IsUsingGlasses'].fillna('')
df_applicant['MaritalStatus'] = df_applicant['MaritalStatus'].fillna('')
df_applicant['Nationality'] = df_applicant['Nationality'].fillna('')
df_applicant['Strengthness'] = df_applicant['Strengthness'].fillna('')
df_applicant['Weaknesses'] = df_applicant['Weaknesses'].fillna('')
df_applicant['TypeOfVehicle'] = df_applicant['TypeOfVehicle'].fillna('')

df_applicant['Height'] = df_applicant['Height'].apply(lambda x: int(abs(x)))
df_applicant['DiseaseHistory'] = df_applicant['DiseaseHistory'].map(str.lower)

df_applicant['IsAbleToWorkRemote'] = pd.get_dummies(df_applicant['IsAbleToWorkRemote']).iloc[:, 1]
df_applicant['CityID'] = pd.to_numeric(df_applicant['CityID'], downcast='integer')
df_applicant['ProvinceID'] = pd.to_numeric(df_applicant['ProvinceID'], downcast='integer')

df_applicant = df_applicant.replace(
    ['', '-', 'none', 'tidak', 'tidak ada', 'gak ada', 'gk ada', 0], 
    np.NaN
).set_index(['ApplicantID']).dropna(how='all')

df_applicant = df_applicant.replace(np.NaN, 0)

df_applicant['Age'] = pd.to_datetime(
    df_applicant.Dob.apply(lambda x: np.NaN if (int(x[:4]) < 1940 or int(x[:4]) > 2004) else x)
).apply(lambda x: get_age(x)).replace(np.NaN, 0).astype(int)
df_applicant.drop(columns=['Dob'], inplace=True)

In [506]:
df_applicant_education['MajorID'] = df_applicant_education['MajorID'].fillna(0)
df_applicant_education['DateStart'] = df_applicant_education['DateStart'].map(pick_date)
df_applicant_education['DateEnd'] = df_applicant_education['DateEnd'].map(pick_date)
df_applicant_education['EducationInstituteName'] = df_applicant_education['EducationInstituteName'].map(str.lower)

df_applicant_education['DateStart'] = df_applicant_education['DateStart'].apply(lambda x: remove_badyears(x, 1970, 2023))
df_applicant_education['DateEnd'] = df_applicant_education['DateEnd'].apply(lambda x: remove_badyears(x, 1980, 2023))
df_applicant_education['DateEnd'] = pd.to_datetime(df_applicant_education['DateEnd']).apply(lambda x: np.NaN if x > datetime.now() else x)
df_applicant_education['DateStart'] = pd.to_datetime(df_applicant_education['DateStart'])
df_applicant_education['YearsOfEducation'] = (df_applicant_education['DateEnd'] - df_applicant_education['DateStart']) / np.timedelta64(1, 'Y')
df_applicant_education.drop(columns=['DateStart', 'DateEnd'], inplace=True)

df_applicant_education.drop_duplicates(subset=['ApplicantID'], keep='last', inplace=True)
df_applicant_education.drop(columns=['ApplicantEducationID'], inplace=True)

In [507]:
# fill null values with yesterday's date
df_applicant_experience['DateTo'] = df_applicant_experience['DateTo'].fillna(str(datetime.now()-timedelta(days=1)))
# pick year from dates
df_applicant_experience['DateFrom'] = df_applicant_experience['DateFrom'].map(pick_date)
df_applicant_experience['DateTo'] = df_applicant_experience['DateTo'].map(pick_date)
# change weird years to zero so it will be easy to remove data
df_applicant_experience['DateFrom'] = df_applicant_experience['DateFrom'].apply(lambda x: years_to_zero(x))
df_applicant_experience['DateTo'] = df_applicant_experience['DateTo'].apply(lambda x: years_to_zero(x))
# lower
df_applicant_experience['Industry'] = df_applicant_experience['Industry'].map(str.lower)
df_applicant_experience['CompanyName'] = df_applicant_experience['CompanyName'].map(str.lower)
# fill null values with "-"
df_applicant_experience['JobDescription'] = df_applicant_experience['JobDescription'].fillna('-')
df_applicant_experience['JobDescription'] = df_applicant_experience['JobDescription'].map(str.lower)
df_applicant_experience['JobDescription'] = df_applicant_experience['JobDescription'].map(html_cleaner)
# TODO: drop bad date values

# lower
df_applicant_experience['Position'] = df_applicant_experience['Position'].map(str.lower)

# df_applicant_experience['ExperienceTime'] = get_experience_time(df_applicant_experience['DateTo'], df_applicant_experience['DateFrom'])

df_applicant_experience = df_applicant_experience[df_applicant_experience['DateFrom'].apply(lambda x: int(x[:4])) > 1990]
df_applicant_experience = df_applicant_experience[df_applicant_experience['DateTo'].apply(lambda x: int(x[:4])) > 1990]

df_applicant_experience = df_applicant_experience.drop(columns=['ApplicantExperienceID'])

df_applicant_experience['DateFrom'] = pd.to_datetime(df_applicant_experience['DateFrom'])
df_applicant_experience['DateTo'] = pd.to_datetime(df_applicant_experience['DateTo'])

df_applicant_experience['MonthsExperience'] = round( abs(df_applicant_experience.DateTo - df_applicant_experience.DateFrom) / np.timedelta64(1, 'M') )
df_applicant_experience.drop(columns=['DateFrom', 'DateTo'], inplace=True)

df_applicant_experience = df_applicant_experience.groupby(['ApplicantID']).agg({
    'Industry': ' '.join ,
    'CompanyName': ' '.join,
    'JobDescription': ' '.join,
    'Position': ' '.join,
    'Salary': 'mean',
    'MonthsExperience': 'sum',
})

# df_applicant_experience['YearsExperience'] = round( abs(df_applicant_experience.DateTo - df_applicant_experience.DateFrom) / np.timedelta64(1, 'Y') )


In [515]:
df_applicant_document = df_applicant_document.groupby(['ApplicantID']).agg(' '.join)
df_applicant_certificate = df_applicant_certificate.groupby(['ApplicantID']).agg(' '.join)

### Merge Dataframe

In [518]:
df_merged = pd.merge(df_applicant, df_applicant_experience, on='ApplicantID')
df_merged = pd.merge(df_merged, df_applicant_certificate, on='ApplicantID')
df_merged = pd.merge(df_merged, df_applicant_education, on='ApplicantID')
df_merged = pd.merge(df_merged, df_applicant_document, on='ApplicantID')
df_merged = pd.merge(df_merged, df_pipeline, on='ApplicantID')
df_merged = pd.merge(df_merged, df_stage, on='StageID')

In [526]:
df_merged.drop(columns=['PipelineID', 'JobID', 'StageID'], inplace=True)

In [543]:
df_merged[df_merged.Label == 'Hired']

,ApplicantID,DiseaseHistory,ExpectedSalary,IsAbleToWorkRemote,CityID,ProvinceID,DriverLicenseType,Gender,IsUsingGlasses,Height,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Age,Industry,CompanyName,JobDescription,Position,Salary,MonthsExperience,Description,EducationInstituteName,Score,EducationLevelID,MajorID,Degree,YearsOfEducation,DocumentName,Label
2863,13,no,1.0,0.0,1.0,1.0,C,Male,0,180.0,Single,Indonesia,swot,swot,Sepeda Motor,0,rudi,mr.,------,it,123.0,0.0,tes,stts,10.00,3,3.0,False,0.084875,Surat Ijazah,Hired
2864,134,0,0.0,1.0,1.0,1.0,C,Female,True,156.0,Single,Indonesia,"Responsible, thorough, having many skills","Having fierce face, lack ability in chit chat,...",Sepeda Motor,30,law services,sidabukke clan & associates,i'm responsible to finish legal task given by ...,legal staff,0.0,33.0,Special Education for Advocate Profession,universitas brawijaya,3.48,2,6.0,True,4.000082,Certificate of Bachelor degree and Academic Tr...,Hired
2865,531,0,4500000.0,1.0,1.0,1.0,C,Female,True,156.0,Single,Indonesia,"Responsible, thorough, having many skills","Having fierce face, lack ability in chit chat,...",Sepeda Motor,30,law services,kantor hukum sidabukke clan & associates,i'm responsible to finish legal task given by ...,legal staff,3300000.0,33.0,Special Education for Advocate Profession Bach...,universitas brawijaya,3.48,2,6.0,True,4.000082,Certificate of Employment Academic Transcript ...,Hired
2866,1599,0,4000000.0,0.0,1.0,1.0,A,Male,0,172.0,Single,INDONESIA,EASILY BLEND WITH THE SITUATION,LESS ATTENTION FOR MYSELF,Mobil,31,event organizer insurance,insight prudential,memastikan penjualan sesuai targetmengawasi da...,team leader marketing (agent),1850000.0,80.0,S1,smpn 35,27.00,7,7.0,False,2.918609,FOTO IJAZAH TRANSKRIP TOEFL CV SURAT LAMARAN,Hired
2867,1599,0,4000000.0,0.0,1.0,1.0,A,Male,0,172.0,Single,INDONESIA,EASILY BLEND WITH THE SITUATION,LESS ATTENTION FOR MYSELF,Mobil,31,event organizer insurance,insight prudential,memastikan penjualan sesuai targetmengawasi da...,team leader marketing (agent),1850000.0,80.0,S1,smpn 35,27.00,7,7.0,False,2.918609,FOTO IJAZAH TRANSKRIP TOEFL CV SURAT LAMARAN,Hired
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3018,39596,maag,4700000.0,1.0,28.0,9.0,C,Male,True,168.0,Single,Indonesia,"pengalaman bekerja yang saya miliki, mengerjak...","kurang teliti, tetapi saya memiliki solusi dal...",Sepeda Motor,25,retail / merchandise jasa advertising telekomu...,pt. dinar sukses abadi pt. multi bagus global ...,1. memonitoring jaringan\r\n2. melakukan penge...,it staff drafter graphic designer it support,3325000.0,67.0,Analis Program,stmik nusa mandiri,3.52,2,0.0,True,4.000082,CV,Hired
3019,39626,0,6000000.0,0.0,1.0,1.0,A,Male,True,170.0,Single,Indonesia,"Excellent legal problem solving, advanced engl...","too detail, overthink, too easy to accept some...",Mobil,26,hukum,kantor advokat abdul salam & associates,contract draftinglegal consultant of pt kejaya...,associate,5000000.0,56.0,Pelatihan Pembuatan Kontrak (Contract Drafting...,universitas surabaya,3.18,2,6.0,True,4.000082,CV and Education Certificates,Hired
3020,39690,0,7000000.0,0.0,421.0,33.0,0,Male,0,150.0,Married,Indonesia,"Semangat bekerja, bisa berkomunikasi dengan ba...",Tidak mau menunda pekerjaan,Mobil,29,lembaga pemerintahan lembaga hukum,kejaksaan negeri manado kantor advocad lukas &...,"memeriksa berkas perkara, membuat dakwaan, mel...",asisten jaksa paraligel,7500000.0,84.0,Sertifikat kelulusan Pendidikan Khusus Profesi...,universitas kristen indonesia tomohon,3.50,2,6.0,True,3.671533,KTP Ijasah Transkrip Nilai Sartifikat Foto,Hired
3021,39690,0,7000000.0,0.0,421.0,33.0,0,Male,0,150.0,Married,Indonesia,"Semangat bekerja, bisa berkomunikasi dengan ba...",Tidak mau menunda pekerjaan,Mobil,29,lembaga pemerintahan lembaga hukum,kejaksaan negeri manado kantor advocad lukas &...,"memeriksa berkas perkara, membuat dakwaan, mel...",asist

In [540]:
df_merged[df_merged.Label == 'Rejected'].shape

(2863, 31)

In [535]:
df_merged[df_merged.ApplicantID.duplicated()]

,ApplicantID,DiseaseHistory,ExpectedSalary,IsAbleToWorkRemote,CityID,ProvinceID,DriverLicenseType,Gender,IsUsingGlasses,Height,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Age,Industry,CompanyName,JobDescription,Position,Salary,MonthsExperience,Description,EducationInstituteName,Score,EducationLevelID,MajorID,Degree,YearsOfEducation,DocumentName,Label
6,124,0,4000000.0,0.0,3.0,1.0,C,Male,0,169.0,Married,Indonesia,bekerja di depan komputer berjam jam,0,Sepeda Motor,39,bunker service transportir transportasi / log...,pt. pelayaran hub maritim indonesia pt. andala...,1. pengurusan perijinan perusahaan2. pelaporan...,staff operasional / legal manager operasional ...,2.283333e+06,115.0,Mengikuti Kegiatan komputer,bhayangkara surabaya,2.97,2,6.0,True,4.829668,Pengalaman Kerja Ijazah SI Transkrip Nilai,Rejected
7,124,0,4000000.0,0.0,3.0,1.0,C,Male,0,169.0,Married,Indonesia,bekerja di depan komputer berjam jam,0,Sepeda Motor,39,bunker service transportir transportasi / log...,pt. pelayaran hub maritim indonesia pt. andala...,1. pengurusan perijinan perusahaan2. pelaporan...,staff operasional / legal manager operasional ...,2.283333e+06,115.0,Mengikuti Kegiatan komputer,bhayangkara surabaya,2.97,2,6.0,True,4.829668,Pengalaman Kerja Ijazah SI Transkrip Nilai,Rejected
8,124,0,4000000.0,0.0,3.0,1.0,C,Male,0,169.0,Married,Indonesia,bekerja di depan komputer berjam jam,0,Sepeda Motor,39,bunker service transportir transportasi / log...,pt. pelayaran hub maritim indonesia pt. andala...,1. pengurusan perijinan perusahaan2. pelaporan...,staff operasional / legal manager operasional ...,2.283333e+06,115.0,Mengikuti Kegiatan komputer,bhayangkara surabaya,2.97,2,6.0,True,4.829668,Pengalaman Kerja Ijazah SI Transkrip Nilai,Rejected
11,267,0,3000000.0,1.0,1.0,1.0,C,Male,True,172.0,Single,INDONESIA,Memiliki Loyalitas Tinggi Terhadap Pekerjaan,Kurang terhadap pekerjaan berbau statistika,Sepeda Motor,31,outsourcing,pt gos indoraya,"setup, troubleshooting, maintenance hardware d...",it support,3.800000e+06,19.0,Sertifikat BNSP Desain dan Pemrograman Website,universitas narotama,3.00,2,4.0,True,2.748859,CV Hendy Agha Satya Ramadan Referensi Kerja SK...,Rejected
12,267,0,3000000.0,1.0,1.0,1.0,C,Male,True,172.0,Single,INDONESIA,Memiliki Loyalitas Tinggi Terhadap Pekerjaan,Kurang terhadap pekerjaan berbau statistika,Sepeda Motor,31,outsourcing,pt gos indoraya,"setup, troubleshooting, maintenance hardware d...",it support,3.800000e+06,19.0,Sertifikat BNSP Desain dan Pemrograman Website,universitas narotama,3.00,2,4.0,True,2.748859,CV Hendy Agha Satya Ramadan Referensi Kerja SK...,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3465,35467,0,8000000.0,0.0,2.0,2.0,C,Male,0,167.0,Single,Indonesia,mempunyai semangat yang tinggi dan merupakan o...,tidak sabar dalam melakukan sesuatu,Sepeda Motor,28,jasa hukum,3r advokat,drafting yang berkaitan dengan perkara litigas...,senior associate lawyer,8.000000e+06,59.0,Pendidikan Khusus Profesi Advokat Ujian Profes...,universitas surabaya,3.36,2,6.0,False,4.000082,CV Ijazah Sertifikat PKPA Sertifikat Ujian Adv...,Phone Interview
3468,36621,not avaliable,5500000.0,0.0,87.0,3.0,A,Male,True,176.0,Single,Indonesia,"Work Diligently, Quickly Learn, Accuracy, Disc...","Critic, Easy to Joke, Forgetful",Mobil,29,pharmacy garment,pt. bernofarm pharmaceutical pt. bina busana i...,work experience:&nbsp;legal drafting/legal con...,legal officer legal staff,5.250000e+06,83.0,TOEFL,universitas pancasila,3.27,2,6.0,True,4.583256,IJAZAH CERTIFICATE TOEFL CERTIFICATE COMPUTER,Phone Interview
3474,44883,tidak ada,0.0,0.0,139.0,1.0,C,Female,0,162.0,Married,Indonesia,"Komunikatif,mampu menganalisa masalah dan meng...",Tidak mudah percaya,Motor,36,conveyor siostem & mesin industri distributor ...,"pt. multi karya tehnik pt. lautan luas , tbk s...","- melakukan recruitmen , melakukan seleksi cal...",hr personalia & general affair customer 

In [529]:
df_merged.head(2)

,ApplicantID,DiseaseHistory,ExpectedSalary,IsAbleToWorkRemote,CityID,ProvinceID,DriverLicenseType,Gender,IsUsingGlasses,Height,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Age,Industry,CompanyName,JobDescription,Position,Salary,MonthsExperience,Description,EducationInstituteName,Score,EducationLevelID,MajorID,Degree,YearsOfEducation,DocumentName,Label
0,3,0,4000000.0,0.0,1.0,1.0,C,Male,0,178.0,Single,WNI,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Sepeda Motor,29,telekomunikasi retail service,pt icool international pt dbl indonesia pt may...,- maintenance dealer (cek stock dealer apakah ...,sales executive whole sales credit marketing o...,3.666667e+06,22.0,Klub basket Atmajaya termasuk yang paling seri...,atmajaya yogyakarta university,3.01,2,7.0,True,4.503857,CV Fernando Martin Lukas,Rejected
1,13,no,1.0,0.0,1.0,1.0,C,Male,0,180.0,Single,Indonesia,swot,swot,Sepeda Motor,0,rudi,mr.,------,it,1.230000e+02,0.0,tes,stts,10.00,3,3.0,False,0.084875,Surat Ijazah,Rejected


In [527]:
df_merged[df_merged.Nationality.str.contains('nesia')]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [465]:
df_merged.Nationality.unique()

array(['indonesia'], dtype=object)

In [421]:
df_merged.head()

,ApplicantID,DiseaseHistory,ExpectedSalary,IsAbleToWorkRemote,CityID,ProvinceID,DriverLicenseType,Gender,IsUsingGlasses,Height,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Age,Industry,CompanyName,JobDescription,Position,Salary,MonthsExperience,Description,EducationInstituteName,Score,EducationLevelID,MajorID,Degree,YearsOfEducation,DocumentName,PipelineID,JobID,StageID,Label
0,3,0,4000000.0,0.0,1.0,1.0,C,Male,0,178.0,Single,WNI,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Sepeda Motor,29,telekomunikasi retail service,pt icool international pt dbl indonesia pt may...,- maintenance dealer (cek stock dealer apakah ...,sales executive whole sales credit marketing o...,3.666667e+06,22.0,Klub basket Atmajaya termasuk yang paling seri...,atmajaya yogyakarta university,3.01,2,7.0,True,4.503857,CV Fernando Martin Lukas,1,6,10,Rejected
1,13,no,1.0,0.0,1.0,1.0,C,Male,0,180.0,Single,Indonesia,swot,swot,Sepeda Motor,0,rudi,mr.,------,it,1.230000e+02,0.0,tes,stts,10.00,3,3.0,False,0.084875,Surat Ijazah,7248,1192,10,Rejected
2,25,0,5000000.0,0.0,1.0,1.0,C,Male,0,160.0,Single,Indonesia,"Pekerja keras, mandiri, bertanggung jawab, dap...",sedikit pelupa,Sepeda Motor,33,software house software house,pt hiro technology pt tempo inti media,"trouble shooting computer, programmer, mainten...",junior progammer progammer and analyst,4.500000e+06,34.0,Juara 1 SMA 1 Pace IA,budi luhur jakarta,3.53,1,4.0,True,2.086285,CV,13,8,10,Rejected
3,69,0,4000000.0,0.0,1.0,1.0,C,Female,0,155.0,Single,Indonesia,"kerja keras, tidak mudah putus asa","suka menunda pekerjaan, pendiam",Sepeda Motor,28,digital printing konsultasi pendidikan,pelangi komputer psikologi unair,- -,operator komputer tester,5.250000e+05,15.0,Skor total : 470,sma n 2 pare,8.50,5,0.0,False,3.000746,Ijazah Traanskrip Nilai CV Sertifikat TOEFL SK...,24,16,10,Rejected
4,97,0,3850000.0,0.0,1.0,1.0,C,Male,0,165.0,Single,Indonesia,"Have good adaptation and communication skills,...",Do not like to work with paln that turns in th...,Sepeda Motor,29,consulting retail / ekspedisi / furniture / pr...,anima konsultan malang nusantara surya sakti (...,- freeline job- melakukan rekrutmen dan seleks...,freeline tester hrd r&a off,2.425000e+06,18.0,English for spesific purpose,universitas islam negeri maulana malik ibrahim...,3.53,2,3.0,True,4.668131,Profile,34,16,10,Rejected


In [360]:
df_merged[df_merged.Nationality.astype(str).str.contains('nesia')]

,ApplicantID,DiseaseHistory,ExpectedSalary,IsAbleToWorkRemote,CityID,ProvinceID,DriverLicenseType,Gender,IsUsingGlasses,Height,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Age,Industry,CompanyName,JobDescription,Position,Salary,MonthsExperience,PipelineID,JobID,StageID,Label
2,13,no,1.0,0.0,1.0,1.0,C,Male,0,180.0,Single,Indonesia,swot,swot,Sepeda Motor,0,rudi,mr.,------,it,123.0,0.0,7248,1192,10,Rejected
4,21,0,4500000.0,0.0,1.0,1.0,C,Male,True,162.0,Single,Indonesia,0,0,Sepeda Motor,29,it it,pt gaea global solusi cv isocorp technology,scope of responsibility :analysing and develop...,java web developer android developer (native),3300000.0,42.0,12,3,10,Rejected
5,25,0,5000000.0,0.0,1.0,1.0,C,Male,0,160.0,Single,Indonesia,"Pekerja keras, mandiri, bertanggung jawab, dap...",sedikit pelupa,Sepeda Motor,33,software house software house,pt hiro technology pt tempo inti media,"trouble shooting computer, programmer, mainten...",junior progammer progammer and analyst,4500000.0,34.0,13,8,10,Rejected
6,29,0,6000000.0,0.0,1.0,1.0,C,Female,0,160.0,Single,Indonesia,"Tekun, Mandiri, jujur","Ambisius, Pesimis, Lupa",Sepeda Motor,30,developer/property,abdael nusa group,membuat rab,qs,4000000.0,103.0,15,7,10,Rejected
7,62,2900000,4000000.0,0.0,1.0,1.0,C,Female,True,161.0,Single,Indonesia,"bekerja dengan jujur dan kerja keras, bisa dia...",sering lupa makan siang kalo sedang bekerja,Sepeda Motor,28,pendidikan,smk st. louis,mengelola surat masuk dan keluar beserta ekspe...,tata usaha,2900000.0,127.0,19,16,10,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8019,44860,batu empedu dan kolesterol,0.0,0.0,1.0,1.0,C,Male,0,155.0,Married,Indonesia,Individu yg Kritis Logis dan Metodis,"Individu yg risk avoider, butuh info lengkap d...",Sepeda Motor,45,bank hsbc,hsbc,keuangan owner,asst. manager,7000000.0,49.0,24810,3045,2,Job Offered
8020,1979,"cough, fever and light flu",4500000.0,1.0,2.0,2.0,C,Male,0,172.0,Single,Indonesia,"work in under pressure, high dedication and a ...",sometimes less neat in appearance and excessiv...,Sepeda Motor,28,contractor consultant design,pt. cipta selaras cv. hanif engineering,i assisted qc staff for maked&nbsp;a quality c...,internship program student freelance engineeri...,400000.0,13.0,702,13,5,Examination
8021,8072,nothing,4300000.0,1.0,1.0,1.0,C,Female,0,155.0,Single,Indonesia,"Mampu bekerja dengan cepat dan tepat, teliti, ...","Kurang dalam public speaking, grogi dengan lin...",Sepeda Motor,26,real estate supplier alat listrik,pt. sinar megah raya (brighton) pt. sentral gl...,1. membuat perjanjian jual beli atau perjanjia...,finance staff admin staff,2900000.0,50.0,17956,1616,5,Examination
8022,36883,0,7000000.0,0.0,0.0,1.0,C,Male,0,170.0,Married,Indonesia,"Adaptasi yg cepat, jujur, tanggung jawab",Mudah terbawa emosi,Sepeda Motor,32,kayu otomotif pelayaran konsultan pajak,pt. widjaja putra jaya pt. surya timur sakti j...,administrasi akuntasiadministrasi pajakadminis...,accounting finance accounting finance ar offic...,4450000.0,99.0,17137,1565,5,Examination


In [345]:
df_merged.Nationality.unique()

array(['WNI', 'INDONESIA', 'Indonesia', 0, 'indonesia', 'Indonesian', 'W',
       'Warga Negara Indonesia', 'WARGA NEGARA INDONESIA', 'Indonesia ',
       'jawa/ java', 'INDONESIA ', 'indonesian', 'INDONESIAN',
       'indonesia ', 'INdonesia', 'Warga Negara Indonesia ', 'JAKARTA',
       'Sundanese', 'Indoneaia', 'Batak', 'INDONESIA (WNI)',
       'Warganegara Indonesia', 'Jawa', 'iNDONESIA', 'wni', 'Indonesja',
       'INDONDESIA', 'Minangkabau', 'Indonesia / WNI', 'WNl',
       'Indonesia (WNI)', 'iIndonesia', 'Indonesia jawa',
       'Warga Negara indonesia', 'warga negara indonesia ', 'indoensia',
       'Indonesisa', 'NKRI', 'Warga negra Indonesia', 'Wni',
       'Indonesian Citizen', 'javanese', 'Indonesis', 'Indoensia',
       'Idonesia', 'WNi', 'Warga negara indonesia', 'Java',
       'Warga negara Indonesia', 'Republic of Indonesia',
       'Indonesian - Javanese', 'Imdonesia', 'Indonesian ', 'JAVA',
       'Warna Negara Indonesia', 'MELAYU', 'WNI Keturunan', 'WNI ',
       '

In [342]:
df_merged.head()

,ApplicantID,DiseaseHistory,ExpectedSalary,IsAbleToWorkRemote,CityID,ProvinceID,DriverLicenseType,Gender,IsUsingGlasses,Height,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Age,Industry,CompanyName,JobDescription,Position,Salary,MonthsExperience,PipelineID,JobID,StageID,Label
0,3,0,4000000.0,0.0,1.0,1.0,C,Male,0,178.0,Single,WNI,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Sepeda Motor,29,telekomunikasi retail service,pt icool international pt dbl indonesia pt may...,- maintenance dealer (cek stock dealer apakah ...,sales executive whole sales credit marketing o...,3.666667e+06,22.0,1,6,10,Rejected
1,10,0,3700000.0,0.0,1.0,1.0,C,Male,0,168.0,Single,INDONESIA,"KONSISTEN, MAMPU BEKERJA DENGAN TEAM",TERLALU TAAT DENGAN KATA KONSISTEN,Sepeda Motor,29,perhotelan event organizer,country heritage hotel surabaya victory event ...,bertugas sebagai receptionist sekaligus merang...,front desk agent - night audit marketing - adm...,3.200000e+06,109.0,3,1,10,Rejected
2,13,no,1.0,0.0,1.0,1.0,C,Male,0,180.0,Single,Indonesia,swot,swot,Sepeda Motor,0,rudi,mr.,------,it,1.230000e+02,0.0,7248,1192,10,Rejected
3,18,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0,0,0,0,0,pemerintahan pendidikan pendidikan,badan pusat statistik kabupaten trenggalek bim...,1.membuat\r\nberkas sensus pemutakhiran basis ...,staff bagian statistik administrasi dan tentor...,1.200000e+06,42.0,9,2,10,Rejected
4,21,0,4500000.0,0.0,1.0,1.0,C,Male,True,162.0,Single,Indonesia,0,0,Sepeda Motor,29,it it,pt gaea global solusi cv isocorp technology,scope of responsibility :analysing and develop...,java web developer android developer (native),3.300000e+06,42.0,12,3,10,Rejected


In [238]:
df_merged.head()

,ApplicantID,DiseaseHistory,Dob,ExpectedSalary,IsAbleToWorkRemote,CityID,ProvinceID,DriverLicenseType,Gender,IsUsingGlasses,Height,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,ApplicantExperienceID,DateFrom,DateTo,Industry,CompanyName,JobDescription,Position,Salary,PipelineID,JobID,StageID
0,3,none,1994-01-06,4000000.0,0,1,1,C,Male,False,178,Single,WNI,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Sepeda Motor,1,2016-04-01,2017-04-01,telekomunikasi,pt icool international,- maintenance dealer (cek stock dealer apakah ...,sales executive,3000000.0,1,6,10
1,3,none,1994-01-06,4000000.0,0,1,1,C,Male,False,178,Single,WNI,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Sepeda Motor,2,2017-04-01,2017-07-01,retail,pt dbl indonesia,- melakukan kerjasama dengan pihak sekolah unt...,whole sales,3800000.0,1,6,10
2,3,none,1994-01-06,4000000.0,0,1,1,C,Male,False,178,Single,WNI,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Sepeda Motor,3,2017-08-01,2018-03-01,service,pt maybank indonesia finance,- melakukan analisa kelayakan kredit mobil kon...,credit marketing officer,4200000.0,1,6,10
3,10,-,1993-10-02,3700000.0,0,1,1,C,Male,False,168,Single,INDONESIA,"KONSISTEN, MAMPU BEKERJA DENGAN TEAM",TERLALU TAAT DENGAN KATA KONSISTEN,Sepeda Motor,4,2016-12-01,2023-03-23,perhotelan,country heritage hotel surabaya,bertugas sebagai receptionist sekaligus merang...,front desk agent - night audit,3600000.0,3,1,10
4,10,-,1993-10-02,3700000.0,0,1,1,C,Male,False,168,Single,INDONESIA,"KONSISTEN, MAMPU BEKERJA DENGAN TEAM",TERLALU TAAT DENGAN KATA KONSISTEN,Sepeda Motor,5,2014-02-01,2016-11-01,event organizer,victory event organizer,bertugas sebagai marketing yang juga merangkap...,marketing - administrasi,2800000.0,3,1,10


In [ ]:
df_merged[['ApplicantID', 'JobDescription', 'Position', 'Label']]

In [102]:
filter_rows_by_values(df_merged, 'Label', ['Hired', 'Rejected'])

,ApplicantID,Strengthness,Weaknesses,FirstName,LastName,ApplicantExperienceID,DateFrom,DateTo,Industry,CompanyName,JobDescription,Position,Salary,PipelineID,JobID,StageID,Label
0,3,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Edo,,1,2016-04-01,2017-04-01,telekomunikasi,pt icool international,- maintenance dealer (cek stock dealer apakah ...,sales executive,3000000.0,1,6,10,Rejected
1,3,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Edo,,2,2017-04-01,2017-07-01,retail,pt dbl indonesia,- melakukan kerjasama dengan pihak sekolah unt...,whole sales,3800000.0,1,6,10,Rejected
2,3,"Pekerja Keras,Teliti, Ambisius","Pelupa,Tidak cepat tanggap",Edo,,3,2017-08-01,2018-03-01,service,pt maybank indonesia finance,- melakukan analisa kelayakan kredit mobil kon...,credit marketing officer,4200000.0,1,6,10,Rejected
3,10,"KONSISTEN, MAMPU BEKERJA DENGAN TEAM",TERLALU TAAT DENGAN KATA KONSISTEN,Pratama,Surya,4,2016-12-01,2023-03-23,perhotelan,country heritage hotel surabaya,bertugas sebagai receptionist sekaligus merang...,front desk agent - night audit,3600000.0,3,1,10,Rejected
4,10,"KONSISTEN, MAMPU BEKERJA DENGAN TEAM",TERLALU TAAT DENGAN KATA KONSISTEN,Pratama,Surya,5,2014-02-01,2016-11-01,event organizer,victory event organizer,bertugas sebagai marketing yang juga merangkap...,marketing - administrasi,2800000.0,3,1,10,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13579,40043,memiliki komitmen tinggi dalam sebuah pekerjaa...,"Menyukai hal-hal detail, Mudah kecewa karena g...",Wahyudi Robby,Sutanto,16654,2015-01-01,2020-06-01,software house,pt kliklabs indokreasi,pertama\r\nsaya bekerja sebagai game programme...,senior programmer & project manager,8000000.0,18314,1618,11,Hired
13580,40043,memiliki komitmen tinggi dalam sebuah pekerjaa...,"Menyukai hal-hal detail, Mudah kecewa karena g...",Wahyudi Robby,Sutanto,16655,2013-04-01,2014-12-01,erp program,pt gbs indonesia,saya\r\nbertugas untuk mengembangkan program e...,programmer,2000000.0,18314,1618,11,Hired
13581,43028,None,None,Ardian,Arifin,16661,2018-01-01,2020-07-01,outsouring,pt. sumber abadi karya,sebagai driver direksi,driver,0.0,21908,2799,11,Hired
13582,43756,"public speaking, persuasif, kreatif, negosiasi...",mudah percaya dengan orang,Vanesha Emanuela,Barimbing,16662,2019-09-01,2023-03-23,otomotif,pt surya sudeco,"handling team, prospecting, dealing new custom...",supervisor marketing,8000000.0,23191,2918,11,Hired


### Check Null Values

In [ ]:
df_merged.isna().sum()

ApplicantID                 0
Strengthness             1722
Weaknesses               2100
FirstName                   0
LastName                    0
ApplicantExperienceID       0
DateFrom                    0
DateTo                      0
Industry                    0
CompanyName                 0
JobDescription              0
Position                    0
Salary                      0
dtype: int64